## Microsoft Security Incident Prediction


Este dataset brinda una oportunidad para manejar información de seguridad basada en datos reales, incluyendo alertas y evidencias asociadas, con el objetivo de estimar con exactitud la severidad de los incidentes y las medidas de remediación sugeridas. La solución se enfoca en apoyar a los centros de operaciones de seguridad (SOC) en su proceso de toma de decisiones, mejorando los tiempos de respuesta y minimizando errores en la administración de amenazas.

## Inicializacion de Spark

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = SparkSession.builder \
    .appName("MuestreoSeguridad") \
    .getOrCreate()

In [2]:
df = spark.read.csv(r"C:\Users\luis_\Downloads\archive\GUIDE_Train.csv", header=True, inferSchema=True)

In [3]:
df.printSchema()

root
 |-- Id: long (nullable = true)
 |-- OrgId: integer (nullable = true)
 |-- IncidentId: integer (nullable = true)
 |-- AlertId: integer (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- DetectorId: integer (nullable = true)
 |-- AlertTitle: integer (nullable = true)
 |-- Category: string (nullable = true)
 |-- MitreTechniques: string (nullable = true)
 |-- IncidentGrade: string (nullable = true)
 |-- ActionGrouped: string (nullable = true)
 |-- ActionGranular: string (nullable = true)
 |-- EntityType: string (nullable = true)
 |-- EvidenceRole: string (nullable = true)
 |-- DeviceId: integer (nullable = true)
 |-- Sha256: integer (nullable = true)
 |-- IpAddress: integer (nullable = true)
 |-- Url: integer (nullable = true)
 |-- AccountSid: integer (nullable = true)
 |-- AccountUpn: integer (nullable = true)
 |-- AccountObjectId: integer (nullable = true)
 |-- AccountName: integer (nullable = true)
 |-- DeviceName: integer (nullable = true)
 |-- NetworkMessageId: i

## Paso 1 Caracterización de la Población

In [4]:
print("\nAnálisis de 'Category':")
df.groupBy("Category").count().orderBy(F.col("count").desc()).show(truncate=False)


Análisis de 'Category':
+-------------------+-------+
|Category           |count  |
+-------------------+-------+
|InitialAccess      |4293041|
|Exfiltration       |1577965|
|SuspiciousActivity |1003933|
|CommandAndControl  |826691 |
|Impact             |750885 |
|CredentialAccess   |300518 |
|Execution          |267594 |
|Malware            |144081 |
|Discovery          |129342 |
|Persistence        |72682  |
|DefenseEvasion     |46662  |
|LateralMovement    |41754  |
|Ransomware         |18974  |
|UnwantedSoftware   |18211  |
|Collection         |14753  |
|PrivilegeEscalation|4671   |
|Exploit            |4648   |
|CredentialStealing |388    |
|WebExploit         |38     |
|Weaponization      |6      |
+-------------------+-------+



In [5]:
print("\nAnálisis de 'IncidentGrade':")
df.groupBy("IncidentGrade").count().orderBy(F.col("count").desc()).show(truncate=False)


Análisis de 'IncidentGrade':
+--------------+-------+
|IncidentGrade |count  |
+--------------+-------+
|BenignPositive|4110817|
|TruePositive  |3322713|
|FalsePositive |2031967|
|NULL          |51340  |
+--------------+-------+



In [6]:
print("\nAnálisis de 'EntityType':")
df.groupBy("EntityType").count().orderBy(F.col("count").desc()).show(truncate=False)


Análisis de 'EntityType':
+-----------------+-------+
|EntityType       |count  |
+-----------------+-------+
|Ip               |2181194|
|User             |1932416|
|MailMessage      |1173154|
|Machine          |699208 |
|File             |688402 |
|Url              |682578 |
|CloudLogonRequest|638565 |
|Mailbox          |483158 |
|Process          |345732 |
|MailCluster      |224684 |
|CloudApplication |216811 |
|CloudLogonSession|212382 |
|RegistryValue    |11209  |
|AzureResource    |8166   |
|RegistryKey      |7277   |
|GenericEntity    |4294   |
|OAuthApplication |2595   |
|Malware          |2580   |
|SecurityGroup    |1518   |
|BlobContainer    |306    |
+-----------------+-------+
only showing top 20 rows



In [7]:
print("\nAnálisis de 'ResourceType':")
df.groupBy("ResourceType").count().orderBy(F.col("count").desc()).show(truncate=False)


Análisis de 'ResourceType':
+------------------------------------+-------+
|ResourceType                        |count  |
+------------------------------------+-------+
|NULL                                |9509762|
|Virtual Machine                     |4146   |
|Key Vault                           |480    |
|App Service                         |404    |
|Azure Arc machine                   |373    |
|Storage Account                     |366    |
|Subscription                        |271    |
|SQL Database                        |226    |
|Azure Database for MySQL Server     |182    |
|Azure Resource                      |157    |
|Virtual Machine Scale Set           |112    |
|API Management Service              |94     |
|Key Vault Secret                    |77     |
|Azure Database for PostgreSQL Server|45     |
|SQL Server                          |41     |
|Azure Cosmos DB Account             |30     |
|Networking                          |14     |
|Synapse SQL Pool              

### 1b. Análisis de OSFamily (si es relevante y códigos interpretables)            

In [8]:
print("\nAnálisis de 'OSFamily':")
df.groupBy("OSFamily").count().orderBy(F.col("count").desc()).show()


Análisis de 'OSFamily':
+--------+-------+
|OSFamily|  count|
+--------+-------+
|       5|9322565|
|       0| 190036|
|       1|   2732|
|       2|   1496|
|       3|      7|
|       4|      1|
+--------+-------+



### 1c. Conteo de Nulos en Variables Clave

In [12]:
print("\nConteo de Nulos en Variables Clave:")
columns_to_check = ["Category", "IncidentGrade", "EntityType", "ResourceType", "OSFamily"]
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c + '_null_count') for c in columns_to_check]).show()


Conteo de Nulos en Variables Clave:
+-------------------+------------------------+---------------------+-----------------------+-------------------+
|Category_null_count|IncidentGrade_null_count|EntityType_null_count|ResourceType_null_count|OSFamily_null_count|
+-------------------+------------------------+---------------------+-----------------------+-------------------+
|                  0|                   51340|                    0|                9509762|                  0|
+-------------------+------------------------+---------------------+-----------------------+-------------------+



## Paso 2: Diseño del Particionamiento

### 2a. Selección de Variables de Particionamiento

In [16]:
partitioning_var1 = "Category"
partitioning_var2 = "IncidentGrade"

### 2b. Cálculo de Proporciones por Combinación

In [17]:
print(f"\nCalculando proporciones para combinaciones de '{partitioning_var1}' y '{partitioning_var2}'...")

total_count = df.count()
print(f"Total de registros en D: {total_count}")

partition_counts = df.groupBy(partitioning_var1, partitioning_var2).count()

partition_proportions = partition_counts.withColumn("proportion", F.col("count") / total_count)
partition_proportions.orderBy(F.col("count").desc()).show(truncate=False)


Calculando proporciones para combinaciones de 'Category' y 'IncidentGrade'...
Total de registros en D: 9516837
+------------------+--------------+-------+--------------------+
|Category          |IncidentGrade |count  |proportion          |
+------------------+--------------+-------+--------------------+
|InitialAccess     |TruePositive  |2266513|0.23815822420831628 |
|Exfiltration      |BenignPositive|1080813|0.11356851020985229 |
|InitialAccess     |BenignPositive|1063722|0.11177264042664595 |
|InitialAccess     |FalsePositive |954880 |0.10033585738623033 |
|CommandAndControl |BenignPositive|563544 |0.059215472535675454|
|SuspiciousActivity|BenignPositive|507279 |0.053303319159506465|
|Exfiltration      |FalsePositive |368866 |0.038759306269509505|
|SuspiciousActivity|TruePositive  |356286 |0.03743743851029496 |
|Impact            |BenignPositive|336977 |0.03540850809990757 |
|Impact            |TruePositive  |242524 |0.025483676982173804|
|Impact            |FalsePositive |171348 |

### 2c. Definición de Reglas de Particionamiento

In [18]:
cond_p1 = (F.col("Category") == 'InitialAccess') & (F.col("IncidentGrade") == 'TruePositive')
cond_p2 = (F.col("Category") == 'Exfiltration') & (F.col("IncidentGrade") == 'BenignPositive')
cond_p3 = (F.col("Category") == 'InitialAccess') & (F.col("IncidentGrade") == 'BenignPositive')
cond_p4 = (F.col("Category") == 'InitialAccess') & (F.col("IncidentGrade") == 'FalsePositive')
cond_p5 = (F.col("Category") == 'CommandAndControl') & (F.col("IncidentGrade") == 'BenignPositive')
cond_p6 = (F.col("Category") == 'SuspiciousActivity') & (F.col("IncidentGrade") == 'BenignPositive')

prop_p1 = 0.23815822420831628
prop_p2 = 0.11356851020985229
prop_p3 = 0.11177264042664595
prop_p4 = 0.10033585738623033
prop_p5 = 0.059215472535675454
prop_p6 = 0.053303319159506465

In [19]:
partition_rules = [
    {
        "name": "P1_InitAccess_TruePos",
        "filter_expr": cond_p1,
        "est_proportion": prop_p1
    },
    {
        "name": "P2_Exfiltration_BenignPos",
        "filter_expr": cond_p2,
        "est_proportion": prop_p2
    },
    {
        "name": "P3_InitAccess_BenignPos",
        "filter_expr": cond_p3,
        "est_proportion": prop_p3
    },
    {
        "name": "P4_InitAccess_FalsePos",
        "filter_expr": cond_p4,
        "est_proportion": prop_p4
    },
    {
        "name": "P5_CmdCtrl_BenignPos",
        "filter_expr": cond_p5,
        "est_proportion": prop_p5
     },
     {
        "name": "P6_SuspAct_BenignPos",
        "filter_expr": cond_p6,
        "est_proportion": prop_p6
     }
]

print("Reglas de particionamiento definidas:")
total_prop_check = 0
for rule in partition_rules:
    print(f"- {rule['name']}: Estimación {rule['est_proportion']:.4%}")
    total_prop_check += rule['est_proportion']
print(f"\nSuma total de proporciones estimadas: {total_prop_check:.4f}")

Reglas de particionamiento definidas:
- P1_InitAccess_TruePos: Estimación 23.8158%
- P2_Exfiltration_BenignPos: Estimación 11.3569%
- P3_InitAccess_BenignPos: Estimación 11.1773%
- P4_InitAccess_FalsePos: Estimación 10.0336%
- P5_CmdCtrl_BenignPos: Estimación 5.9215%
- P6_SuspAct_BenignPos: Estimación 5.3303%

Suma total de proporciones estimadas: 0.6764


## Paso 3: Implementación del Filtrado por Partición

In [21]:
partition_dfs = {} # Diccionario para guardar los dataframes de cada partición

for rule in partition_rules:
    partition_name = rule['name']
    filter_condition = rule['filter_expr']

    print(f"\nFiltrando para la partición: {partition_name}")
    print(f"Condición: {filter_condition}")
    
    partition_df = df.filter(filter_condition)
    partition_dfs[partition_name] = partition_df
    
    print(f"Verificación para {partition_name}:")
    partition_df.show(5, truncate=False)
    count_partition = partition_df.count()
    print(f"Registros encontrados en {partition_name}: {count_partition}")


Filtrando para la partición: P1_InitAccess_TruePos
Condición: Column<'((Category = InitialAccess) AND (IncidentGrade = TruePositive))'>
Verificación para P1_InitAccess_TruePos:
+-------------+-----+----------+-------+-------------------+----------+----------+-------------+---------------+-------------+-------------+--------------+-----------------+------------+--------+------+---------+------+----------+----------+---------------+-----------+----------+----------------+--------------+-----------+-----------------+-----------------+-------------+---------------+------------------+------------+--------+----------+--------------+------------+-----+--------+---------+-----------------+--------------+-----------+-----------+-----+-----+
|Id           |OrgId|IncidentId|AlertId|Timestamp          |DetectorId|AlertTitle|Category     |MitreTechniques|IncidentGrade|ActionGrouped|ActionGranular|EntityType       |EvidenceRole|DeviceId|Sha256|IpAddress|Url   |AccountSid|AccountUpn|AccountObjectId|